# Proyecto 8

## Descripción del proyecto

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

## Objetivo

1. Crear un modelo que escoja el plan correcto.

    2. Desarrollar un modelo con la mayor exactitud posible. El umbral de exactitud es 0.75. 
    
        3. Usar el dataset para comprobar la exactitud.

## Pasos

1. Abrir y examinar archivos.
2. Segmentar los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.
3. Investigar la calidad de diferentes modelos cambiando los hiperparámetros. 
4. Comprobar la calidad del modelo usando el conjunto de prueba.
5. Hacer prueba de cordura al módelo. 

## Descripción de los datos

Cada observación en el dataset contiene información del comportamiento mensual sobre un usuario. La información dada es la siguiente:

- сalls — número de llamadas,

- minutes — duración total de la llamada en minutos,

- messages — número de mensajes de texto,

- mb_used — Tráfico de Internet utilizado en MB,

- is_ultra — plan para el mes actual (Ultra - 1, Smart - 0).

## Análisis exploratorio de datos

In [ ]:
# Importación de librerías 

import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Importación de modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


In [4]:
# Importación de archivos

#df = pd.read_csv('users_behavior.csv')
df = pd.read_csv('/datasets/users_behavior.csv')

In [5]:
# Información y visualización

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<span style='color:blue'> Comentario: </span>
1. No hay valores ausentes, los tipos de datos son adecuados. 
2. No es necesario analizar duplicados porque son valores numéricos que pueden repetirse muchas veces. 

## Segmentación de datos fuente

In [6]:
# Creación de conjuntos de entrenamiento, validación y prueba 
# Se busca un proporción 3:1:1 

# Se usará la función train_test_split 2 veces: 
# Primera segmentación: 80% entrenamiento + validación y 20% prueba

df_train_valid, df_test = train_test_split(df, test_size=0.2, random_state=54321)

# Segunda segmentación: 75% entrenamiento y 25% validación

df_train, df_valid = train_test_split(df_train_valid, test_size=0.25, random_state=54321)

# Verificación de tamaños de cada segmento

print('Dataset de entrenamiento:', df_train.shape)
print('Dataset de validación:', df_valid.shape)
print('Dataset de prueba:', df_test.shape)


Dataset de entrenamiento: (1928, 5)
Dataset de validación: (643, 5)
Dataset de prueba: (643, 5)


In [7]:
# Separación de características y objetivo en cada dataset
# La columna 'is_ultra' es el objetivo, el resto de columnas son características

# Dataset de entranamiento

df_train_features = df_train.drop(['is_ultra'], axis=1)
df_train_target = df_train['is_ultra']

# Dataset de validación

df_valid_features = df_valid.drop(['is_ultra'], axis=1)
df_valid_target = df_valid['is_ultra']

# Dataset de prueba

df_test_features = df_test.drop(['is_ultra'], axis=1)
df_test_target = df_test['is_ultra']

## Elección de modelos de clasificación

Al tratarse de una tarea de clasficación, se probarán los siguientes modelos de clasificación: arbol de decisión, bosques aleatorios y regresión logística.

In [ ]:
# Árbol de decisión

# Bucle para probar profundidad del árbol desde 1 hasta 10
for depth in range(1,11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)

    # Entrenamiento
    model.fit(df_train_features, df_train_target)

    # Exactitud con set de validación
    score = model.score(df_valid_features, df_valid_target)

    print(f"max_depth: {depth}, score: {score}")

max_depth: 1, score: 0.776049766718507
max_depth: 2, score: 0.7931570762052877
max_depth: 3, score: 0.8087091757387247
max_depth: 4, score: 0.8040435458786936
max_depth: 5, score: 0.8164852255054432
max_depth: 6, score: 0.8102643856920684
max_depth: 7, score: 0.8149300155520995
max_depth: 8, score: 0.8118195956454122
max_depth: 9, score: 0.8149300155520995
max_depth: 10, score: 0.7931570762052877


<span style='color:blue'> Comentario </span>

El mejor score de exactitud del model de **árbol aleatorio** se obtuvo con una profundidad de 5:

        max_depth: 5, score: 0.8164852255054432

In [ ]:
# Bosque aleatorio

# Bucle para probar diferentes valores del hiperparámetro n_estimators, desde 1 hasta 10
for est in range(1,11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    
    # Entrenamiento
    model.fit(df_train_features, df_train_target)

    # Exactitud con set de validación
    score = model.score(df_valid_features, df_valid_target)

    print(f"n_estimators = {est}, exactitud = {score}")

n_estimators = 1, exactitud = 0.7262830482115086
n_estimators = 2, exactitud = 0.7822706065318819
n_estimators = 3, exactitud = 0.7729393468118196
n_estimators = 4, exactitud = 0.8133748055987559
n_estimators = 5, exactitud = 0.80248833592535
n_estimators = 6, exactitud = 0.8102643856920684
n_estimators = 7, exactitud = 0.8102643856920684
n_estimators = 8, exactitud = 0.8242612752721618
n_estimators = 9, exactitud = 0.8118195956454122
n_estimators = 10, exactitud = 0.8149300155520995


<span style='color:blue'> Comentario </span>

El mejor score en el modelo de **bosque aleatorio** se obtuvo con n_estimators = 8:

        n_estimators = 8, exactitud = 0.8242612752721618

In [ ]:
# Regresión logística

model = LogisticRegression(random_state=12345, solver='liblinear')

# Entrenamiento del modelo
model.fit(df_train_features, df_train_target)

# Exactitud del modelo entranado con los datos de validación
score = model.score(df_valid_features, df_valid_target)

print('Exactitud del modelo:', score)

Exactitud del modelo: 0.776049766718507


<span style='color:blue'> Comentario </span>

La exactitud del módelo de **regresión logística** fue:

       Exactitud del modelo: 0.7325038880248833 


<span style='color:green'> Elección de modelo </span>

De acuerdo al valor de exactitud en el set de validación, el mejor modelo es de **bosque aleatorio** con un **score** de 0.8242 y **n_estimators** de 8

## Comprobación de calidad del modelo

Se seleccionó el modelo de clasificación de **bosque aleatorio**, por obtener la mejor calificación de exactitud en el set de validación. 

In [11]:
# Prueba del modelo de bosque aleatorio con el set de prueba

model = RandomForestClassifier(random_state=12345, n_estimators=8)

# Entrenamiento
model.fit(df_train_features, df_train_target)

# Exactitud con set de prueba
score = model.score(df_test_features, df_test_target)

print('La exactitud del modelo en el set de prueba es de:', score)

La exactitud del modelo en el set de prueba es de: 0.7651632970451011


<span style='color:blue'> Comentario </span>

La exactitud del modelo con el set de prueba fue de 0.76

        La exactitud del modelo en el set de prueba es de: 0.7651632970451011


## Prueba de cordura

In [12]:
# Modificación del tipo de datos en el target, cambiando los valores numéricos:
# 1 - Ultra, 0 - Smart

df['is_ultra'].replace(1, 'Ultra', inplace=True)
df['is_ultra'].replace(0, 'Smart', inplace=True)

df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,Smart
1,85.0,516.75,56.0,22696.96,Smart
2,77.0,467.66,86.0,21060.45,Smart
3,106.0,745.53,81.0,8437.39,Ultra
4,66.0,418.74,1.0,14502.75,Smart


In [13]:
# Creación de sets de entrenamiento, validación y prueba

df_train_valid, df_test = train_test_split(df, test_size=0.2, random_state=54321)

# Segunda segmentación: 75% entrenamiento y 25% validación

df_train, df_valid = train_test_split(df_train_valid, test_size=0.25, random_state=54321)

In [14]:
# Separación de características y objetivo en cada dataset
# La columna 'is_ultra' es el objetivo, el resto de columnas son características

# Dataset de entranamiento

df_train_features = df_train.drop(['is_ultra'], axis=1)
df_train_target = df_train['is_ultra']

# Dataset de validación

df_valid_features = df_valid.drop(['is_ultra'], axis=1)
df_valid_target = df_valid['is_ultra']

# Dataset de prueba

df_test_features = df_test.drop(['is_ultra'], axis=1)
df_test_target = df_test['is_ultra']

In [15]:
# Prueba de cordura
# Entrenamiento

model = RandomForestClassifier(random_state=12345, n_estimators=8)

model.fit(df_train_features, df_train_target)

# Prueba

score = model.score(df_test_features, df_test_target)

print('Exactitud del modelo en la prueba de cordura:', score)

Exactitud del modelo en la prueba de cordura: 0.7651632970451011


<span style='color:blue'> Comentario </span>

Se obtuvo una exactitud de 0.7651 con la prueba de cordura:

       Exactitud del modelo en la prueba de cordura: 0.7651632970451011

La exactitud del modelo después de cambiar los valores numéricos de la columna target ('is_ultra') por los strings respectivos (Ultra y Smart), fue igual al score obtenido antes del cambio. Por lo tanto, el modelo funciona bien. 

## Conclusiones

1. Se evaluaron 3 modelos de clasificación, probando diferentes hiperparámetros en cada uno: 
    - árbol de decisión
    - bosque aleatorio
    - regresión logística
2. El modelo de bosque aleatorio con n_estimators igual a 8 obtuvo el mejor score, igual a 0.824
3. La exactitud del modelo de bosque aleatorio con el conjunto de prueba fue de 0.765. El umbral de exactitud solicitado era de 0.75, por lo que el modelo creado es aceptable. 
4. La prueba de cordura consistió en cambiar los valores target, de numérico (0 y 1) a su descripción textual (Smart y Ultra). El score de exactitud fue igual al primer score obtenido. Se puede decir que la prueba de cordura fue superada. 